
# 🎯 Projeto de Transcrição e Análise de Sentimentos

Este projeto utiliza **Python e NLP** para transcrever áudios e analisar sentimentos de interações com clientes.  
A análise é baseada no **Net Promoter Score (NPS)** para entender melhor a satisfação dos clientes.

## 🔍 Estrutura do Código:
1. **Importação das bibliotecas**
2. **Carregamento e limpeza dos dados**
3. **Processamento e análise**
4. **Visualização dos resultados**


In [1]:
!pip install openai-whisper
!pip install whisper
!pip install SpeechRecognition
!pip install --upgrade whisper
!pip install --upgrade librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s

In [2]:
#importando as bibliotecas
import whisper
import re
from google.colab import files
import os
import markdown
import warnings
import pickle
import time
import concurrent.futures
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Criar diretório para salvar transcrições
os.makedirs('transcricoes', exist_ok=True)

# Criar diretório para salvar cache
os.makedirs('cache', exist_ok=True)

# Módulo de transcrição e análise de áudio
def transcrever_e_analisar_audio(arquivo_de_audio):
    try:
        print(f"Transcrição em progresso para: {arquivo_de_audio}...")
        modelo = whisper.load_model("small")
        resultado = modelo.transcribe(arquivo_de_audio)
        print(f"Transcrição concluída para: {arquivo_de_audio}!")

        texto = resultado["text"]
        duracao = resultado["segments"][-1]["end"]  # Duração do áudio em segundos

        # Melhorar a formatação
        texto = markdown.markdown(texto)

        # Remover ruídos
        texto = re.sub(r"Tá|Ah", "", texto)

        # Calcular total de palavras
        total_palavras = len(re.findall(r'\w+', texto))

        return texto, duracao, total_palavras
    except Exception as e:
        print(f"Erro ao transcrever áudio: {e}")
        return None, None, None

# Função para salvar a transcrição em um arquivo
def salvar_transcricao(texto, nome_arquivo):
    with open(f'transcricoes/{nome_arquivo}', 'w', encoding='utf-8') as f:
        f.write(texto)

# Função para carregar cache
def carregar_cache():
    try:
        with open('cache/cache.pkl', 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return {}

# Função para salvar cache
def salvar_cache(cache):
    with open('cache/cache.pkl', 'wb') as f:
        pickle.dump(cache, f)

# Selecionar arquivo de áudio
def selecionar_arquivos():
    uploaded = files.upload()
    arquivos_de_audio_uploadados = []
    for arquivo_de_audio in uploaded.keys():
        if arquivo_de_audio.endswith('.mp3') or arquivo_de_audio.endswith('.wav'):
            arquivos_de_audio_uploadados.append(arquivo_de_audio)
    print("Arquivos de áudio uploadados:")
    for i, arquivo in enumerate(arquivos_de_audio_uploadados):
        print(f"{i+1}. {arquivo}")
    return arquivos_de_audio_uploadados

# Função para processar e salvar transcrição de um único arquivo de áudio
def processar_audio(arquivo_de_audio):
    if arquivo_de_audio in cache:
        print(f"Arquivo de áudio '{arquivo_de_audio}' já foi baixado e transcrito. Usando cache...")
        texto, duracao, total_palavras = cache[arquivo_de_audio]
    else:
        texto, duracao, total_palavras = transcrever_e_analisar_audio(arquivo_de_audio)
        cache[arquivo_de_audio] = (texto, duracao, total_palavras)
        salvar_cache(cache)

    if texto:
        texto_formatado = re.sub(r'([.!?])', r'\1\n', texto)

        # Salvar a transcrição em um arquivo
        nome_arquivo = arquivo_de_audio + '_transcricao.txt'
        salvar_transcricao(texto_formatado, nome_arquivo)
        print(f"Transcrição salva em: {nome_arquivo}")

        # Retornar dados necessários para o CSV
        return arquivo_de_audio.split('.')[0], texto, duracao, total_palavras
    else:
        print(f"Erro ao transcrever o áudio {arquivo_de_audio}.")
        return None, None, None, None

# Main
cache = carregar_cache()
arquivos_de_audio_uploadados = selecionar_arquivos()

if arquivos_de_audio_uploadados:
    resultados = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for arquivo_de_audio in arquivos_de_audio_uploadados:
            resultado = processar_audio(arquivo_de_audio)
            if resultado[0]:  # Se houver transcrição
                resultados.append(resultado)

    # Criar DataFrame e salvar em CSV
    df = pd.DataFrame(resultados, columns=['id_audio', 'transcricao', 'tempo_de_chamada', 'total_palavras'])
    df.to_csv('transcricoes/transcricoes.csv', index=False)
    print("CSV gerado com sucesso!")
else:
    print("Erro ao carregar o arquivo de áudio.")


Saving 2874774.wav to 2874774.wav
Saving 2874830.wav to 2874830.wav
Saving 2961972.wav to 2961972.wav
Saving 2962046.wav to 2962046.wav
Saving 2962074.wav to 2962074.wav
Arquivos de áudio uploadados:
1. 2874774.wav
2. 2874830.wav
3. 2961972.wav
4. 2962046.wav
5. 2962074.wav
Transcrição em progresso para: 2874774.wav...


100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 51.5MiB/s]


Transcrição concluída para: 2874774.wav!
Transcrição salva em: 2874774.wav_transcricao.txt
Transcrição em progresso para: 2874830.wav...
Transcrição concluída para: 2874830.wav!
Transcrição salva em: 2874830.wav_transcricao.txt
Transcrição em progresso para: 2961972.wav...
Transcrição concluída para: 2961972.wav!
Transcrição salva em: 2961972.wav_transcricao.txt
Transcrição em progresso para: 2962046.wav...
Transcrição concluída para: 2962046.wav!
Transcrição salva em: 2962046.wav_transcricao.txt
Transcrição em progresso para: 2962074.wav...
Transcrição concluída para: 2962074.wav!
Transcrição salva em: 2962074.wav_transcricao.txt
CSV gerado com sucesso!


In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Função para processar texto
def processar_texto(texto):
    lemmatizer = WordNetLemmatizer()
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)  # Remove números
    palavras = texto.split()
    palavras = [palavra for palavra in palavras if palavra not in stopwords.words('portuguese')]
    palavras = [lemmatizer.lemmatize(palavra) for palavra in palavras]
    return ' '.join(palavras)

# Função para rotular automaticamente com base nas palavras-chave
def rotular_transcricao(texto, limiar_promotor=1, limiar_detrator=1):
    contagem_promotor = contar_palavras(texto, palavras_promotor)
    contagem_detrator = contar_palavras(texto, palavras_detrator)
    total_promotor = sum(contagem_promotor.values())
    total_detrator = sum(contagem_detrator.values())

    if total_promotor >= limiar_promotor and total_promotor > total_detrator:
        return "promotor"
    elif total_detrator >= limiar_detrator and total_detrator > total_promotor:
        return "detrator"
    else:
        return "neutro"

# Função para contar palavras de destaque
def contar_palavras(texto, palavras):
    contagem = {palavra: texto.lower().count(palavra) for palavra in palavras}
    return contagem

# Função para realizar a rotulagem
def realizar_rotulagem(diretorio):
    df = pd.read_csv(os.path.join(diretorio, 'transcricoes.csv'))
    rotulos = []

    for index, row in df.iterrows():
        texto_processado = processar_texto(row['transcricao'])
        rotulo = rotular_transcricao(texto_processado)
        rotulos.append(rotulo)

    # Adicionar rótulos ao DataFrame e salvar em CSV
    df['rotulo'] = rotulos
    df.to_csv(os.path.join(diretorio, 'transcricoes_rotuladas.csv'), index=False)
    print("\nRótulos adicionados e CSV atualizado!")

# Definir palavras de destaque para promotores e detratores
palavras_promotor = [
    "ótimo", "excelente", "muito bom", "bom", "satisfeito", "recomendo",
    "gostei", "amor", "paixão", "orgulho", "satisfação", "facilidade", "eficiência",
    "boa experiência", "produto incrível", "suporte excelente"
]
palavras_detrator = [
    "ruim", "péssimo", "muito ruim", "desapontado", "insatisfeito", "não recomendo",
    "não gostei", "ódio", "desgosto", "desapontamento", "descontentamento",
    "dificuldade", "inconveniente", "problema", "erro", "falta de suporte"
]

# Realizar a rotulagem
realizar_rotulagem('transcricoes')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...



Rótulos adicionados e CSV atualizado!


In [4]:
import pandas as pd
from google.colab import files

df = pd.read_csv('transcricoes/transcricoes_rotuladas.csv')
df['tempo_de_chamada'] = df['tempo_de_chamada'].astype(int)


# Salvar o DataFrame concatenado em um novo CSV com separador ';'
df = df.to_csv('transcricoes/concatenado.csv', sep=';', index=False)

# Baixar o arquivo concatenado
files.download('transcricoes/concatenado.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
df  = pd.read_csv('transcricoes/concatenado.csv', sep=';')
df.head()

,id_audio,transcricao,tempo_de_chamada,total_palavras,rotulo
0,2874774,"<p>Candinavia, Natalia, bom dia. Bom dia, Nata...",301,627,promotor
1,2874830,"<p>Bom dia, eu falo com o Sr. Pedro. Bom dia, ...",354,827,promotor
2,2961972,"<p>Alô, bom dia. Alô, bom dia. Alô? Alô, bom d...",563,1486,promotor
3,2962046,"<p>Depende de Cláudia. Oi Cláudia, bom dia. Lí...",177,401,detrator
4,2962074,"<p>Alô, Alô. Fala com o Sr. Pedro Rogério. Iss...",661,1605,detrator
